In [1]:
##### Building conda environment #############################
# conda create --name greta2022 python=3.9
# pip install --upgrade pip
# pip install jupyter numpy matplotlib pandas

##### Importing function #####################################
import os, sys, time

sys.path

import numpy as np

import matplotlib
matplotlib.use('PDF')
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

import pandas as pd

[SMA data archive](https://lweb.cfa.harvard.edu/cgi-bin/sma/smaarch.pl)

[Splatalogue Line Table](https://splatalogue.online/)

 Go to `Advanced` -> `Select Species`(for example, the molar mass of `CH3OH vt=0-2` is 32), go to `Specify Ranges` to. specify *frequency range* (e.g., 190-400 GHz) and *Energy Range* (e.g., Eu = 0-800 K), go to `Line List Display` and just check *JPL* or *CDMS* (i.e., uncheck all of the rest). And then click **Search**. Then scroll down to the bottom can get you a tab-separated ASCII file.

In [88]:
class smaarchive:
    
    def __init__(self, 
                 outfigname = 'outfig.pdf',
                 figsize = (8, 3),
                 panel_bound  = [0.08, 0.16, 0.88, 0.75],
                 lable_fontsize = 12,
                 xlim    = (180, 350),
                 ylim    = None
                ):
        self.panel_bound = panel_bound
        
        self.outfigname = outfigname
        fig     = plt.figure(figsize = figsize )
        self.ax = fig.add_axes(panel_bound)
        self.ax.tick_params(axis='both', which='major', labelsize=lable_fontsize)

        xtitle = 'Frequency [GHz]'
        ytitle = 'Year - 2000'
        plt.ylabel(ytitle, size = lable_fontsize)
        plt.xlabel(xtitle, size = lable_fontsize)
        
        plt.xlim( [xlim[0], xlim[1]] )
        self.ylim = ylim

    
    def __del__(self):
        pass
    
    ########################################################################################
    #     Methods
    ########################################################################################
    
    def load_archive(self, filename = '', 
                    ):
        self.DataFrame = pd.read_csv(filename, delim_whitespace=True, #sep=' ', 
                                     usecols = [0, 5, 6, 7, 8],
                                     header=None, 
                                     names=["year", "LO", "num_BL", "AngRes", "TOS"]
                                    )
        
        
        
    def load_linetb(self, filename = ''):
        print('Processing line table: ', filename)
        self.line_tb = pd.read_csv(filename, 
                                    delim_whitespace=True, #sep=' ', 
                                    header=None, skiprows=1,
                                    usecols = [2, 5, 6],
                                    names=["restfreq", "CDMS_intensity", "Eu_K"]
                                    )
        
        
        
    def plot_linetb(self,
                    color = (0,0,0,1), linewidth = 0.5,
                    intensity_threshold = -999.0,
                    Eu_K_min = 0.0, Eu_K_max = 1e10
                   ):

        num_lines = len( self.line_tb['restfreq'] ) 
        
        for line_id in range(0, num_lines):
            if_plot = False
            x = [self.line_tb['restfreq'][line_id], self.line_tb['restfreq'][line_id]]
            try:
                EuK = float( self.line_tb['Eu_K'][line_id] )
                height = ( (EuK - Eu_K_min) / (Eu_K_max - Eu_K_min) ) * self.ylim[1]
            except:
                print("format problem line id:", line_id, 'EuK:', self.line_tb['Eu_K'][line_id])
                height = self.panel_bound[3] * self.ylim[1]
            
            y = [self.ylim[0], height]
            
            if float(self.line_tb['CDMS_intensity'][line_id]) > intensity_threshold:
                # print( float(self.line_tb['CDMS_intensity'][line_id]) )
                if_plot = True
            
            if if_plot == True:
                plt.plot(
                         x, y, 
                         color = color, linewidth = linewidth
                        )
                
                
                
    def plot_linelegend(self, linetb_files, linecolor_dict, linelabel_dict):
        x = 0.8
        y = self.panel_bound[1] + self.panel_bound[3]
        
        for file in linetb_files:
            label = linelabel_dict[file]
            color = linecolor_dict[file]
            plt.plot([x+0.02,x+0.07],[y+0.015,y+0.015], color=color, linewidth=0.5, transform=self.ax.transAxes)
            self.ax.text(x+0.08, y, 
                         label, color=color, fontsize=6,
                         transform=self.ax.transAxes)
            y-=0.05
            
        
        
    def plot_archive(self,
                    ):
        
        color_dict = {}
        color_dict['sub'] = (1, 0.5, 1.0, 1)
        color_dict['com'] = (0.3, 0.3, 1.0, 1)
        color_dict['ext'] = (0.2, 0.8, 0.2, 1)
        color_dict['vex'] = (1, 0.3, 0.2, 1)
        
        # plotting configuration
        yloc = self.panel_bound[1] + self.panel_bound[3]
        x = np.array( [0.1, 0.2, 0.2, 0.1]) - 0.08
        y = np.array([yloc, yloc, yloc+0.05, yloc+0.05]) + 0.03
        
        for config in ['sub','com','ext','vex']:
            self.ax.text(x[1]+0.005,y[1], ' : '+config, 
                         color = color_dict[config],
                         fontsize=12, transform=self.ax.transAxes)
            self.ax.fill(x, y,
                            edgecolor=color_dict[config],
                            facecolor=None, fill=False, linewidth=1.5, hatch='x',
                            transform=self.ax.transAxes
                            # color=color_dict[source]
                           )
            x += 0.2 
        
        num_obs = len(self.DataFrame['year'])
        print('number of observations: ', num_obs)
        
        for obs_id in range(0, num_obs):
            for sideband in ['usb', 'lsb']:
                x, y = self.getIF( self.DataFrame['year'][obs_id], 
                                   self.DataFrame['LO'][obs_id],
                                   sideband=sideband,
                                   y_min = float(self.DataFrame['year'][obs_id]),
                                   y_max = float(self.DataFrame['year'][obs_id])+1.0,
                                  )
                config = self.getconfig(
                                        self.DataFrame['LO'][obs_id],
                                        self.DataFrame['AngRes'][obs_id],
                                        )
 
                self.ax.fill(x, y,
                        edgecolor=color_dict[config],
                        facecolor=None, fill=False, linewidth=1.5, hatch='x'
                        # color=color_dict[source]
                       )
        
        # set plotting limit
        if self.ylim == None:
            max_year = np.max( self.DataFrame['year'] )
            min_year = np.min( self.DataFrame['year'] )
            plt.ylim( min_year-1, max_year + 8)
            self.ylim = (min_year, max_year)
        else:
            plt.ylim( ylim[0], ylim[1])
        
        plt.show()
        
        
        
    def exportfig(self):
        plt.savefig(self.outfigname, 
                    transparent = True
                   )
        
        
        
    def getconfig(self, LO, AngRes):
        LO = float(LO)
        AngRes = float(AngRes)
        
        if AngRes <= ( 0.9 * (230.0/LO) ):
            return 'vex'
        if ( AngRes <= ( 1.8 * (230.0/LO) ) ) and ( AngRes > ( 0.9 * (230.0/LO) ) ):
            return 'ext'
            
        if ( AngRes <= ( 4.0 * (230.0/LO) ) ) and ( AngRes > ( 1.8 * (230.0/LO) ) ):
            return 'com'
        
        if AngRes > ( 4.0 * (230.0/LO) ):
            return 'sub'
        
        
        
    def getIF(self, year, LO, sideband, y_min=0.0, y_max=1.0):
        year = float(year)
        LO   = float(LO)
        
        if year <= 17:
            if sideband == 'usb':
                if_l, if_u = 4, 8
                
            if sideband == 'lsb':
                if_l, if_u = -8, -4
         
        if year > 17:
            if sideband == 'usb':
                if_l, if_u = 4, 12
                
            if sideband == 'lsb':
                if_l, if_u = -12, -4
        
        x = [LO+if_l, LO+if_u, LO+if_u, LO+if_l]
        y = [y_min, y_min, y_max, y_max]
        
        return x, y
        

In [89]:
linecolor_dict = {}
linelabel_dict     = {}
linetb_files  = []

filename = 'CDMS_ch3oh_190to400GHz_0to800K.tsv'
linetb_files.append(filename)
linecolor_dict[filename] = (0.3,0.3,0,0.5)
linelabel_dict[filename] = 'CH$_{3}$OH v$_{t}$=0-2'

filename = 'CDMS_ch3cch_190to400GHz_0to800K.tsv'
linetb_files.append(filename)
linecolor_dict[filename] = (0.7,0.1,0.2,0.75)
linelabel_dict[filename] = 'CH$_{3}$CCH'

data         = 'g31_sma_345GHz'
archive_file = data + '.txt'
outfigname   = data + '.pdf'

archive = smaarchive(
                     outfigname = outfigname, 
                     xlim    = (180, 400)
                    )

archive.load_archive(filename = archive_file)
archive.plot_archive()

for linetb_file in linetb_files:
    archive.load_linetb(filename = linetb_file)
    archive.plot_linetb(
                        color = linecolor_dict[linetb_file], linewidth=0.2,
                        intensity_threshold = -4,
                        Eu_K_min = 0.0, Eu_K_max = 800.0
                        )
    
archive.plot_linelegend(linetb_files, linecolor_dict, linelabel_dict)


archive.exportfig()

number of observations:  14
Processing line table:  CDMS_ch3oh_190to400GHz_0to800K.tsv
format problem line id: 150 EuK: b
format problem line id: 151 EuK: b
format problem line id: 253 EuK: 1.3b
format problem line id: 254 EuK: 1.3b
format problem line id: 258 EuK: b
format problem line id: 265 EuK: 2.1b
format problem line id: 268 EuK: b
format problem line id: 269 EuK: b
format problem line id: 271 EuK: b
format problem line id: 275 EuK: 1.2b
format problem line id: 884 EuK: b
format problem line id: 885 EuK: b
format problem line id: 900 EuK: b
format problem line id: 902 EuK: 1.05b
format problem line id: 909 EuK: b
format problem line id: 910 EuK: b
format problem line id: 915 EuK: 5.4b
format problem line id: 916 EuK: 5.4b
format problem line id: 917 EuK: 5.4b
format problem line id: 1025 EuK: b
format problem line id: 1033 EuK: 4.80b
format problem line id: 1043 EuK: n.r.
format problem line id: 1067 EuK: 0.41b
format problem line id: 1071 EuK: b
format problem line id: 1073 EuK

/var/folders/df/c3483wdx3j116_smbvb9h5dr0000gn/T/ipykernel_77465/1674501887.py:159: UserWarning: Matplotlib is currently using pdf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
